In [1]:
# import useful library and tools
import numpy as np
import pandas as pd
import os
import seaborn as sns
from matplotlib import pyplot as plt
import scipy
from scipy.stats import ttest_ind
import scipy.stats as stats

from mne_connectivity.viz import plot_connectivity_circle
import json
import argparse
import pandas as pd
import holoviews as hv
from holoviews import opts, dim
import pycirclize

# hv.extension('bokeh')
hv.extension('matplotlib')
hv.output(fig='svg', size=200)

from connectiviz.calculating_connectivity import generate_edges_and_weights, generate_edges_and_weights_with_network, threshold_dataframe, count_network_pairs_above_threshold
from connectiviz.utils import rename_columns_by_region

/Users/jpillai/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/jpillai/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [2]:
orig_timeseries = pd.read_csv('./data/rois_timeseries.csv', header=None, index_col=False)
orig_timeseries = orig_timeseries.drop(orig_timeseries.index[232])
orig_timeseries

,0,1,2,3,4,5,6,7,8,9,...,549,550,551,552,553,554,555,556,557,558
0,1,47.231638,70.670433,62.613936,7928.996515,7938.583471,7939.084116,7929.074852,7911.476335,7892.269206,...,7877.855585,7877.157491,7876.860666,7877.896065,7881.356860,7887.510485,7895.369942,7902.797846,7907.484710,7908.151039
1,2,41.668277,71.035427,62.835749,8177.341861,8184.753358,8182.506566,8170.343082,8151.830654,8132.901934,...,8136.746966,8133.880697,8130.823657,8128.861455,8128.899718,8130.971888,8134.195364,8137.235424,8139.035762,8139.405802
2,3,54.244406,75.285714,62.576592,8973.608273,8977.205698,8971.068113,8956.736038,8939.204948,8924.924179,...,8929.657952,8932.782167,8935.103830,8936.202487,8936.681207,8937.631496,8939.543043,8942.035874,8944.019196,8944.366881
3,4,33.908163,76.030612,61.634694,8104.080322,8109.561682,8107.586334,8098.324892,8084.899976,8072.179801,...,8085.452063,8086.855103,8089.763197,8093.096716,8095.680206,8096.698241,8095.741277,8092.946778,8088.868605,8084.317920
4,5,50.676364,87.721818,55.578182,9687.032519,9687.574672,9678.245010,9661.795304,9643.720740,9629.966858,...,9641.437018,9642.602069,9642.552707,9640.915494,9638.627621,9637.260376,9637.906764,9640.563158,9644.155799,9647.160930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,242,38.860656,49.409836,39.868852,9713.941758,9709.531954,9707.599373,9708.146120,9710.219246,9712.403564,...,9741.201388,9740.820741,9741.147037,9741.261158,9740.543801,9739.008881,9737.250708,9735.892270,9735.099001,9734.390365
241,243,50.160494,51.728395,42.098765,9937.806888,9948.032151,9948.640800,9937.730936,9917.995835,9895.707200,...,9911.678618,9917.584687,9922.871410,9925.773645,9925.406340,9921.964838,9916.467469,9910.313980,9904.887692,9901.275143
242,244,40.566667,55.766667,42.783333,10150.347327,10152.896252,10158.275285,10164.217594,10168.197811,10168.582023,...,10153.095325,10154.473572,10156.048966,10156.173100,10154.694478,10152.971261,10152.918880,10155.578304,10160.244312,10164.661218
243,245,48.380952,54.559524,36.250000,9873.596064,9886.757382,9895.355911,9895.252139,9886.214042,9871.833484,...,9839.848830,9851.935152,9865.788400,9877.405320,9884.907668,9888.456444,9889.083816,9887.600400,9884.136940,9878.588821


In [3]:
from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(
    kind="correlation"
)

In [15]:
#drop the first column named '0'
orig_timeseries = orig_timeseries.drop(orig_timeseries.columns[0], axis=1) 
orig_timeseries

,1,2,3,4,5,6,7,8,9,10,...,549,550,551,552,553,554,555,556,557,558
0,47.231638,70.670433,62.613936,7928.996515,7938.583471,7939.084116,7929.074852,7911.476335,7892.269206,7877.819383,...,7877.855585,7877.157491,7876.860666,7877.896065,7881.356860,7887.510485,7895.369942,7902.797846,7907.484710,7908.151039
1,41.668277,71.035427,62.835749,8177.341861,8184.753358,8182.506566,8170.343082,8151.830654,8132.901934,8119.337828,...,8136.746966,8133.880697,8130.823657,8128.861455,8128.899718,8130.971888,8134.195364,8137.235424,8139.035762,8139.405802
2,54.244406,75.285714,62.576592,8973.608273,8977.205698,8971.068113,8956.736038,8939.204948,8924.924179,8918.811602,...,8929.657952,8932.782167,8935.103830,8936.202487,8936.681207,8937.631496,8939.543043,8942.035874,8944.019196,8944.366881
3,33.908163,76.030612,61.634694,8104.080322,8109.561682,8107.586334,8098.324892,8084.899976,8072.179801,8064.602629,...,8085.452063,8086.855103,8089.763197,8093.096716,8095.680206,8096.698241,8095.741277,8092.946778,8088.868605,8084.317920
4,50.676364,87.721818,55.578182,9687.032519,9687.574672,9678.245010,9661.795304,9643.720740,9629.966858,9624.285779,...,9641.437018,9642.602069,9642.552707,9640.915494,9638.627621,9637.260376,9637.906764,9640.563158,9644.155799,9647.160930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,38.860656,49.409836,39.868852,9713.941758,9709.531954,9707.599373,9708.146120,9710.219246,9712.403564,9713.421419,...,9741.201388,9740.820741,9741.147037,9741.261158,9740.543801,9739.008881,9737.250708,9735.892270,9735.099001,9734.390365
241,50.160494,51.728395,42.098765,9937.806888,9948.032151,9948.640800,9937.730936,9917.995835,9895.707200,9877.889778,...,9911.678618,9917.584687,9922.871410,9925.773645,9925.406340,9921.964838,9916.467469,9910.313980,9904.887692,9901.275143
242,40.566667,55.766667,42.783333,10150.347327,10152.896252,10158.275285,10164.217594,10168.197811,10168.582023,10165.260482,...,10153.095325,10154.473572,10156.048966,10156.173100,10154.694478,10152.971261,10152.918880,10155.578304,10160.244312,10164.661218
243,48.380952,54.559524,36.250000,9873.596064,9886.757382,9895.355911,9895.252139,9886.214042,9871.833484,9857.403634,...,9839.848830,9851.935152,9865.788400,9877.405320,9884.907668,9888.456444,9889.083816,9887.600400,9884.136940,9878.588821


In [36]:
orig_timeseries

array([[   47.231638,    70.670433,    62.613936, ...,  7902.797846,
         7907.48471 ,  7908.151039],
       [   41.668277,    71.035427,    62.835749, ...,  8137.235424,
         8139.035762,  8139.405802],
       [   54.244406,    75.285714,    62.576592, ...,  8942.035874,
         8944.019196,  8944.366881],
       ...,
       [   40.566667,    55.766667,    42.783333, ..., 10155.578304,
        10160.244312, 10164.661218],
       [   48.380952,    54.559524,    36.25    , ...,  9887.6004  ,
         9884.13694 ,  9878.588821],
       [   39.394444,    54.122222,    38.883333, ...,  8960.447754,
         8950.700898,  8944.5654  ]])

In [33]:
# calculate correlation of each array with each other in orig_timeseries
correlation_matrix = correlation_measure.fit_transform([orig_timeseries])[0]
correlation_matrix

array([[ 1.00000000e+00, -2.05556706e-02, -1.58460888e-04, ...,
         7.83555650e-02,  7.87322720e-02,  7.88271721e-02],
       [-2.05556706e-02,  1.00000000e+00, -1.38594348e-01, ...,
        -1.92522742e-01, -1.91122934e-01, -1.90860153e-01],
       [-1.58460888e-04, -1.38594348e-01,  1.00000000e+00, ...,
         2.45351084e-01,  2.43978575e-01,  2.42727898e-01],
       ...,
       [ 7.83555650e-02, -1.92522742e-01,  2.45351084e-01, ...,
         1.00000000e+00,  9.74998155e-01,  9.74970463e-01],
       [ 7.87322720e-02, -1.91122934e-01,  2.43978575e-01, ...,
         9.74998155e-01,  1.00000000e+00,  9.75001118e-01],
       [ 7.88271721e-02, -1.90860153e-01,  2.42727898e-01, ...,
         9.74970463e-01,  9.75001118e-01,  1.00000000e+00]])

In [41]:
corr_test = np.corrcoef(orig_timeseries)
corr_test

array([[1.        , 0.99990776, 0.9997604 , ..., 0.99958939, 0.99958657,
        0.99962484],
       [0.99990776, 1.        , 0.99980173, ..., 0.99971427, 0.99965936,
        0.99965202],
       [0.9997604 , 0.99980173, 1.        , ..., 0.99973774, 0.9995462 ,
        0.99957211],
       ...,
       [0.99958939, 0.99971427, 0.99973774, ..., 1.        , 0.99937013,
        0.99935722],
       [0.99958657, 0.99965936, 0.9995462 , ..., 0.99937013, 1.        ,
        0.99966936],
       [0.99962484, 0.99965202, 0.99957211, ..., 0.99935722, 0.99966936,
        1.        ]])

In [32]:
# make correlation_matrix to pandas dataframe
correlation_matrix_df = pd.DataFrame(correlation_matrix)
correlation_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,548,549,550,551,552,553,554,555,556,557
0,1.000000,-0.020556,-0.000158,0.080019,0.079343,0.078255,0.077057,0.076108,0.075678,0.075826,...,0.073977,0.074373,0.074856,0.075444,0.076151,0.076944,0.077724,0.078356,0.078732,0.078827
1,-0.020556,1.000000,-0.138594,-0.184152,-0.182869,-0.184114,-0.187379,-0.191197,-0.193895,-0.194463,...,-0.188948,-0.189975,-0.192070,-0.194357,-0.195801,-0.195771,-0.194410,-0.192523,-0.191123,-0.190860
2,-0.000158,-0.138594,1.000000,0.246006,0.247142,0.247599,0.247155,0.245795,0.243801,0.241703,...,0.240529,0.241339,0.242853,0.244620,0.246054,0.246699,0.246400,0.245351,0.243979,0.242728
3,0.080019,-0.184152,0.246006,1.000000,0.974999,0.974979,0.974948,0.974895,0.974831,0.974788,...,0.974733,0.974742,0.974761,0.974778,0.974794,0.974821,0.974858,0.974885,0.974887,0.974868
4,0.079343,-0.182869,0.247142,0.974999,1.000000,0.975001,0.974967,0.974896,0.974808,0.974743,...,0.974685,0.974694,0.974713,0.974731,0.974751,0.974783,0.974825,0.974854,0.974852,0.974826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,0.076944,-0.195771,0.246699,0.974821,0.974783,0.974798,0.974853,0.974902,0.974919,0.974909,...,0.974816,0.974851,0.974912,0.974969,0.975001,1.000000,0.975001,0.974973,0.974929,0.974889
554,0.077724,-0.194410,0.246400,0.974858,0.974825,0.974837,0.974882,0.974920,0.974927,0.974915,...,0.974813,0.974839,0.974891,0.974943,0.974979,0.975001,1.000000,0.974999,0.974965,0.974926
555,0.078356,-0.192523,0.245351,0.974885,0.974854,0.974863,0.974902,0.974933,0.974936,0.974925,...,0.974838,0.974853,0.974886,0.974920,0.974946,0.974973,0.974999,1.000000,0.974998,0.974970
556,0.078732,-0.191123,0.243979,0.974887,0.974852,0.974859,0.974898,0.974930,0.974937,0.974930,...,0.974866,0.974873,0.974888,0.974899,0.974908,0.974929,0.974965,0.974998,1.000000,0.975001


In [30]:
# Plot the correlation matrix
import numpy as np

from nilearn import plotting

# Make a large figure
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix, 0)
# The labels we have start with the background (0), hence we skip the
# first label
# matrices are ordered for block-like representation
plotting.plot_matrix(
    correlation_matrix
)

In [2]:
#Load in data (input: rois_roi_cor_coef.csv that outputs from the step 3 of IC Pipeline)
data_dir = '/Users/jpillai/Library/CloudStorage/Box-Box/Malu_Hulya_rtFTD-Project/jupyter-notebooks/Controls_FINAL_fMRI/harmonized_data'
rois_roi_cor_coef = pd.read_csv(f'{data_dir}/HARMONIZED_rois_roi_cor_coef.csv', header=None)
rois_roi_cor_coef

,0,1,2,3,4,5,6,7,8,9,...,234,235,236,237,238,239,240,241,242,243
0,1.000000,0.752261,0.267928,0.234015,0.231993,0.185073,0.657539,0.580111,0.564264,0.489354,...,0.226943,0.247075,0.264794,0.269992,0.219746,0.183376,0.178541,0.346992,0.346399,0.424032
1,0.752261,1.000000,0.487045,0.463584,0.501262,0.476522,0.669958,0.559565,0.486774,0.551640,...,0.437841,0.395019,0.334859,0.381798,0.361585,0.272769,0.299424,0.505437,0.423753,0.502379
2,0.267928,0.487045,1.000000,0.625159,0.754652,0.742677,0.251120,-0.019553,-0.042009,0.070100,...,0.417110,0.306180,0.127785,0.102874,0.288242,0.060604,0.302833,0.319231,0.110378,0.202202
3,0.234015,0.463584,0.625159,1.000000,0.621458,0.674823,0.213502,0.225972,-0.109993,0.029818,...,0.399798,0.340793,0.174012,0.355940,0.405288,0.368013,0.433594,0.428512,0.106816,0.186776
4,0.231993,0.501262,0.754652,0.621458,1.000000,0.858175,0.202887,0.127324,0.084886,0.225743,...,0.499260,0.422451,0.254584,0.187552,0.386245,0.140137,0.384295,0.384719,0.325828,0.219395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.183376,0.272769,0.060604,0.368013,0.140137,0.158005,0.191200,0.264082,0.034426,0.118822,...,0.470058,0.463980,0.393822,0.642091,0.449378,1.000000,0.545377,0.544994,0.200495,0.405379
240,0.178541,0.299424,0.302833,0.433594,0.384295,0.311852,0.201325,0.272064,0.104798,0.245403,...,0.621299,0.509592,0.609271,0.493886,0.656079,0.545377,1.000000,0.652779,0.306749,0.464171
241,0.346992,0.505437,0.319231,0.428512,0.384719,0.339683,0.291421,0.388983,0.300462,0.451997,...,0.656898,0.556745,0.544745,0.628396,0.525052,0.544994,0.652779,1.000000,0.488443,0.632210
242,0.346399,0.423753,0.110378,0.106816,0.325828,0.243818,0.348830,0.434335,0.434089,0.508203,...,0.416442,0.385913,0.647658,0.482247,0.460054,0.200495,0.306749,0.488443,1.000000,0.651402


In [3]:
# Load in the subregions_Yeo7networks.csv file from the /data in this repository
df_networks = pd.read_csv('./data/subregions_Yeo7networks.csv')
network_mapping = df_networks.set_index('Label')['Yeo_7network'].to_dict()
df_networks

,Label,subregion_name,region,Yeo_7network,Yeo_17network,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,1,A8m,SFG_L_7_1,6,17,NaN,NaN,NaN,NaN,NaN,Yeo 7 Network,NaN
1,2,A8m,SFG_R_7_1,4,8,NaN,NaN,NaN,NaN,NaN,ID,Network name
2,3,A8dl,SFG_L_7_2,7,16,NaN,NaN,NaN,NaN,NaN,1,Visual
3,4,A8dl,SFG_R_7_2,6,13,NaN,NaN,NaN,NaN,NaN,2,Somatomotor
4,5,A9l,SFG_L_7_3,7,17,NaN,NaN,NaN,NaN,NaN,3,Dorsal Attention
...,...,...,...,...,...,...,...,...,...,...,...,...
239,240,Otha,Tha_R_8_6,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,241,cTtha,Tha_L_8_7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,242,cTtha,Tha_R_8_7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,243,lPFtha,Tha_L_8_8,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Rename the columns to the region names
rois_roi_cor_coef_df = rename_columns_by_region(rois_roi_cor_coef, df_networks)
rois_roi_cor_coef_df

# # Add region column from df_networks to p_values_df
rois_roi_cor_coef_df['region'] = df_networks['region']

# # Ensure 'region' and 'Yeo_17network' columns are at the beginning of t_stats_df
rois_roi_cor_coef_df['Yeo_7network'] = df_networks['Yeo_7network']
cols = ['region', 'Yeo_7network']  # these are the columns you want to move to the front
cols.extend([col for col in rois_roi_cor_coef_df.columns if col not in cols])

# # Reindex the DataFrame with the new column order
rois_roi_cor_coef_df = rois_roi_cor_coef_df[cols]

In [5]:
rois_roi_cor_coef_df

,region,Yeo_7network,SFG_L_7_1,SFG_R_7_1,SFG_L_7_2,SFG_R_7_2,SFG_L_7_3,SFG_R_7_3,SFG_L_7_4,SFG_R_7_4,...,Tha_L_8_4,Tha_R_8_4,Tha_L_8_5,Tha_R_8_5,Tha_L_8_6,Tha_R_8_6,Tha_L_8_7,Tha_R_8_7,Tha_L_8_8,Tha_R_8_8
0,SFG_L_7_1,6,1.000000,0.752261,0.267928,0.234015,0.231993,0.185073,0.657539,0.580111,...,0.226943,0.247075,0.264794,0.269992,0.219746,0.183376,0.178541,0.346992,0.346399,0.424032
1,SFG_R_7_1,4,0.752261,1.000000,0.487045,0.463584,0.501262,0.476522,0.669958,0.559565,...,0.437841,0.395019,0.334859,0.381798,0.361585,0.272769,0.299424,0.505437,0.423753,0.502379
2,SFG_L_7_2,7,0.267928,0.487045,1.000000,0.625159,0.754652,0.742677,0.251120,-0.019553,...,0.417110,0.306180,0.127785,0.102874,0.288242,0.060604,0.302833,0.319231,0.110378,0.202202
3,SFG_R_7_2,6,0.234015,0.463584,0.625159,1.000000,0.621458,0.674823,0.213502,0.225972,...,0.399798,0.340793,0.174012,0.355940,0.405288,0.368013,0.433594,0.428512,0.106816,0.186776
4,SFG_L_7_3,7,0.231993,0.501262,0.754652,0.621458,1.000000,0.858175,0.202887,0.127324,...,0.499260,0.422451,0.254584,0.187552,0.386245,0.140137,0.384295,0.384719,0.325828,0.219395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,Tha_R_8_6,0,0.183376,0.272769,0.060604,0.368013,0.140137,0.158005,0.191200,0.264082,...,0.470058,0.463980,0.393822,0.642091,0.449378,1.000000,0.545377,0.544994,0.200495,0.405379
240,Tha_L_8_7,0,0.178541,0.299424,0.302833,0.433594,0.384295,0.311852,0.201325,0.272064,...,0.621299,0.509592,0.609271,0.493886,0.656079,0.545377,1.000000,0.652779,0.306749,0.464171
241,Tha_R_8_7,0,0.346992,0.505437,0.319231,0.428512,0.384719,0.339683,0.291421,0.388983,...,0.656898,0.556745,0.544745,0.628396,0.525052,0.544994,0.652779,1.000000,0.488443,0.632210
242,Tha_L_8_8,0,0.346399,0.423753,0.110378,0.106816,0.325828,0.243818,0.348830,0.434335,...,0.416442,0.385913,0.647658,0.482247,0.460054,0.200495,0.306749,0.488443,1.000000,0.651402


In [6]:
#Based on a selected threshold, keep the values above or equal to the threshold
threshold = 0.8
thresholded_rois_roi_cor_coef_df = threshold_dataframe(rois_roi_cor_coef_df, threshold)
thresholded_rois_roi_cor_coef_df

#save thresholded_rois_roi_cor_coef_df to csv
thresholded_rois_roi_cor_coef_df.to_csv(f'/Users/jpillai/Documents/rois_roi_cor_coef_thresholded.csv', index=False)

In [7]:
from pycirclize import Circos
from pycirclize.parser import Matrix
import pandas as pd

In [8]:
# # Generate edges and weights for the thresholded DataFrame
edges_weights = generate_edges_and_weights(thresholded_rois_roi_cor_coef_df, ['region', 'Yeo_7network'])
edges_weights

{('SFG_L_7_1', 'SFG_L_7_1'): 0.9999999999999966,
 ('SFG_L_7_1', 'SFG_R_7_1'): 0.0,
 ('SFG_L_7_1', 'SFG_L_7_2'): 0.0,
 ('SFG_L_7_1', 'SFG_R_7_2'): 0.0,
 ('SFG_L_7_1', 'SFG_L_7_3'): 0.0,
 ('SFG_L_7_1', 'SFG_R_7_3'): 0.0,
 ('SFG_L_7_1', 'SFG_L_7_4'): 0.0,
 ('SFG_L_7_1', 'SFG_R_7_4'): 0.0,
 ('SFG_L_7_1', 'SFG_L_7_5'): 0.0,
 ('SFG_L_7_1', 'SFG_R_7_5'): 0.0,
 ('SFG_L_7_1', 'SFG_L_7_6'): 0.0,
 ('SFG_L_7_1', 'SFG_R_7_6'): 0.0,
 ('SFG_L_7_1', 'SFG_L_7_7'): 0.0,
 ('SFG_L_7_1', 'SFG_R_7_7'): 0.0,
 ('SFG_L_7_1', 'MFG_L_7_1'): 0.0,
 ('SFG_L_7_1', 'MFG_R_7_1'): 0.0,
 ('SFG_L_7_1', 'MFG_L_7_2'): 0.0,
 ('SFG_L_7_1', 'MFG_R_7_2'): 0.0,
 ('SFG_L_7_1', 'MFG_L_7_3'): 0.0,
 ('SFG_L_7_1', 'MFG_R_7_3'): 0.0,
 ('SFG_L_7_1', 'MFG_L_7_4'): 0.0,
 ('SFG_L_7_1', 'MFG_R_7_4'): 0.0,
 ('SFG_L_7_1', 'MFG_L_7_5'): 0.0,
 ('SFG_L_7_1', 'MFG_R_7_5'): 0.0,
 ('SFG_L_7_1', 'MFG_L_7_6'): 0.0,
 ('SFG_L_7_1', 'MFG_R_7_6'): 0.0,
 ('SFG_L_7_1', 'MFG_L_7_7'): 0.0,
 ('SFG_L_7_1', 'MFG_R_7_7'): 0.0,
 ('SFG_L_7_1', 'IFG_L_6_1'): 0.0,

In [10]:
# Generate the same edges and weights but based on networks of the pairings of regions
edges_weights_with_networks = generate_edges_and_weights_with_network(thresholded_rois_roi_cor_coef_df, ['region', 'Yeo_7network'], 'Yeo_7network')
edges_weights_with_networks

{(6, 6): 0.9999999999999966,
 (6, 4): 0.0,
 (6, 7): 0.0,
 (6, 3): 0.0,
 (6, 2): 0.0,
 (6, 5): 0.0,
 (6, 1): 0.0,
 (6, 0): 0.0,
 (4, 6): 0.0,
 (4, 4): 0.9999999999999966,
 (4, 7): 0.0,
 (4, 3): 0.0,
 (4, 2): 0.0,
 (4, 5): 0.0,
 (4, 1): 0.0,
 (4, 0): 0.0,
 (7, 6): 0.0,
 (7, 4): 0.0,
 (7, 7): 0.9999999999999966,
 (7, 3): 0.0,
 (7, 2): 0.0,
 (7, 5): 0.0,
 (7, 1): 0.0,
 (7, 0): 0.0,
 (3, 6): 0.0,
 (3, 4): 0.0,
 (3, 7): 0.0,
 (3, 3): 0.9999999999999966,
 (3, 2): 0.0,
 (3, 5): 0.0,
 (3, 1): 0.0,
 (3, 0): 0.0,
 (2, 6): 0.0,
 (2, 4): 0.0,
 (2, 7): 0.0,
 (2, 3): 0.0,
 (2, 2): 0.9999999999999966,
 (2, 5): 0.0,
 (2, 1): 0.0,
 (2, 0): 0.0,
 (5, 6): 0.0,
 (5, 4): 0.0,
 (5, 7): 0.0,
 (5, 3): 0.0,
 (5, 2): 0.0,
 (5, 5): 0.9999999999999966,
 (5, 1): 0.0,
 (5, 0): 0.0,
 (1, 6): 0.0,
 (1, 4): 0.0,
 (1, 7): 0.0,
 (1, 3): 0.0,
 (1, 2): 0.0,
 (1, 5): 0.0,
 (1, 1): 0.9999999999999966,
 (1, 0): 0.0,
 (0, 6): 0.0,
 (0, 4): 0.0,
 (0, 7): 0.0,
 (0, 3): 0.0,
 (0, 2): 0.0,
 (0, 5): 0.0,
 (0, 1): 0.0,
 (0, 0): 0.99

In [11]:
# Create the DataFrame for the chord diagram (contains all regions and their connections)
# connections_df = pd.DataFrame({
#     'source': [edge[0][0] for edge in edges_weights_with_networks],
#     'target': [edge[0][1] for edge in edges_weights_with_networks],
#     'value': [edge[1] for edge in edges_weights_with_networks]
# })

connections_df_networks = pd.DataFrame({
    'source': [edge[0] for edge in edges_weights_with_networks.keys()],
    'target': [edge[1] for edge in edges_weights_with_networks.keys()],
    'value': list(edges_weights_with_networks.values())
})

In [12]:
connections_df = pd.DataFrame({
    'source': [edge[0] for edge in edges_weights.keys()],
    'target': [edge[1] for edge in edges_weights.keys()],
    'value': list(edges_weights.values())
})

In [13]:
connections_df

,source,target,value
0,SFG_L_7_1,SFG_L_7_1,1.0
1,SFG_L_7_1,SFG_R_7_1,0.0
2,SFG_L_7_1,SFG_L_7_2,0.0
3,SFG_L_7_1,SFG_R_7_2,0.0
4,SFG_L_7_1,SFG_L_7_3,0.0
...,...,...,...
59531,Tha_R_8_8,Tha_R_8_6,0.0
59532,Tha_R_8_8,Tha_L_8_7,0.0
59533,Tha_R_8_8,Tha_R_8_7,0.0
59534,Tha_R_8_8,Tha_L_8_8,0.0


In [14]:
# If the value is not 0, set it to 1
connections_df['value'] = connections_df['value'].map(lambda x: 1 if x != 0 else 0)
connections_df

,source,target,value
0,SFG_L_7_1,SFG_L_7_1,1
1,SFG_L_7_1,SFG_R_7_1,0
2,SFG_L_7_1,SFG_L_7_2,0
3,SFG_L_7_1,SFG_R_7_2,0
4,SFG_L_7_1,SFG_L_7_3,0
...,...,...,...
59531,Tha_R_8_8,Tha_R_8_6,0
59532,Tha_R_8_8,Tha_L_8_7,0
59533,Tha_R_8_8,Tha_R_8_7,0
59534,Tha_R_8_8,Tha_L_8_8,0


In [15]:
# If the value is not 0, set it to 1
connections_df_networks['value'] = connections_df_networks['value'].map(lambda x: 1 if x != 0 else 0)
connections_df_networks

,source,target,value
0,6,6,1
1,6,4,0
2,6,7,0
3,6,3,0
4,6,2,0
...,...,...,...
59,0,3,0
60,0,2,0
61,0,5,0
62,0,1,0


pycirclize.circos.Circos

In [16]:
matrix = Matrix.parse_fromto_table(connections_df_networks)
matrix

In [17]:
circos = Circos.initialize_from_matrix(
    matrix,
    space=3,
    cmap="viridis",
    ticks_interval=5,
    label_kws=dict(size=12, r=110),
    link_kws=dict(direction=1, ec="black", lw=0.5),
)

print(connections_df_networks.to_string(index=False))
fig = circos.plotfig()

 source  target  value
      6       6      1
      6       4      0
      6       7      0
      6       3      0
      6       2      0
      6       5      0
      6       1      0
      6       0      0
      4       6      0
      4       4      1
      4       7      0
      4       3      0
      4       2      0
      4       5      0
      4       1      0
      4       0      0
      7       6      0
      7       4      0
      7       7      1
      7       3      0
      7       2      0
      7       5      0
      7       1      0
      7       0      0
      3       6      0
      3       4      0
      3       7      0
      3       3      1
      3       2      0
      3       5      0
      3       1      0
      3       0      0
      2       6      0
      2       4      0
      2       7      0
      2       3      0
      2       2      1
      2       5      0
      2       1      0
      2       0      0
      5       6      0
      5       4      0
      5    

In [19]:
# save the figure
fig.savefig('/Users/jpillai/Documents/circos_plot.png', dpi=300)

In [36]:
# # If the source or target is 0, drop the row
# connections_df = connections_df[connections_df['source'] != 0]
# connections_df = connections_df[connections_df['target'] != 0]
# connections_df

In [68]:
with open('./data/Yeo_7network_names.json', 'r') as file:
    Yeo_7networks_names = json.load(file)

In [69]:
nodes_df = pd.DataFrame(Yeo_7networks_names.items(), columns=['group', 'name'])
nodes = hv.Dataset(nodes_df, 'index')
nodes_df

,group,name
0,1,Visual
1,2,Somatomotor
2,3,Dorsal Attention
3,4,Ventral Attention
4,5,Limbic
5,6,Frontoparietal
6,7,Default


In [70]:
#unique values in the source and target columns
unique_nodes = pd.concat([connections_df['source'], connections_df['target']]).unique()
len(unique_nodes)

244

In [71]:
#if the source and target are the same, drop the row
connections_df = connections_df[connections_df['source'] != connections_df['target']]
connections_df

,source,target,value
1,SFG_L_7_1,SFG_R_7_1,0
2,SFG_L_7_1,SFG_L_7_2,0
3,SFG_L_7_1,SFG_R_7_2,0
4,SFG_L_7_1,SFG_L_7_3,0
5,SFG_L_7_1,SFG_R_7_3,0
...,...,...,...
59530,Tha_R_8_8,Tha_L_8_6,0
59531,Tha_R_8_8,Tha_R_8_6,0
59532,Tha_R_8_8,Tha_L_8_7,0
59533,Tha_R_8_8,Tha_R_8_7,0


In [72]:
connections_df['value'].unique()

array([0, 1])

In [41]:
# # drop the rows where the value is 0
# connections_df = connections_df[connections_df['value'] != 0]
# connections_df

In [73]:
indices = range(244)

# nodes_df = pd.DataFrame({
#     'index': indices,  # The identifiers matching 'source' and 'target' in connections_df
#     'name': unique_nodes    # The names you want to display
# })

# nodes_df

In [74]:
# #replace the names in the source and target columns with the corresponding index
# connections_df = connections_df.replace(unique_nodes, indices)
# connections_df

In [75]:
# nodes = hv.Dataset(nodes_df, 'index')

nodes = hv.Dataset(nodes_df, 'index')
nodes.data

,index,group,name
0,0,1,Visual
1,1,2,Somatomotor
2,2,3,Dorsal Attention
3,3,4,Ventral Attention
4,4,5,Limbic
5,5,6,Frontoparietal
6,6,7,Default


In [76]:
# pip install pycirclize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 1.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [78]:
hv.output(size=400)

# chord = hv.Chord((connections_df, nodes))
chord = hv.Chord(connections_df)
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('source').str(), 
               labels='name', node_color=dim('index').str()))

# Render the chart to an HTML file
hv.save(chord, '/Users/jpillai/Documents/chord_network_0.8.html', backend='matplotlib')

In [83]:
from pycirclize import Circos
from pycirclize.utils import calc_group_spaces, ColorCycler

sectors = {name: 10 for name in "ABCDEFGHIJ"}
group1, group2, group3 = list("ABCD"), list("EFG"), list("HIJ")

# Calculate appropriate group between/within spaces
# In this example, 10 sectors are displayed as groups dividied into [4, 3, 3]
spaces = calc_group_spaces([4, 3, 3], space_bw_group=15, space_in_group=2)
circos = Circos(sectors, space=spaces)

# Plot sector track
ColorCycler.set_cmap("Set3")
for sector in circos.sectors:
    track = sector.add_track(r_lim=(85, 90))
    track.axis(fc=ColorCycler())
    track.text(sector.name)

# Plot group sectors range
ColorCycler.set_cmap("tab10")
for idx, group in enumerate((group1, group2, group3), 1):
    group_deg_lim = circos.get_group_sectors_deg_lim(group)
    circos.rect(r_lim=(97, 100), deg_lim=group_deg_lim, fc=ColorCycler(), ec="black", lw=0.5)
    group_center_deg = sum(group_deg_lim) / 2
    circos.text(f"Group{idx}", r=105, deg=group_center_deg, adjust_rotation=True)

fig = circos.plotfig()

In [84]:
# save the figure 
fig.savefig("/Users/jpillai/Documents/group_sectors.png")